In [75]:
import numpy as np
import pandas as pd
#  Random Search
import yfinance as yf
from sklearn.model_selection import ParameterGrid

In [76]:
df = yf.Ticker('NQ=F')
df = df.history(start='2010-01-01', end='2024-01-01')

### Definimos los rangos de parámetros


In [77]:
sma_1_range = np.arange(10,100,10)
sma_2_range = np.arange(100,300,50)

### Función de backtest para cada combinación de parámetros


In [78]:
def backtest_strategy(sma_1, sma_2):
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()

    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    df.dropna(inplace=True)
    
    return (df['Strategy_Returns'] + 1).prod() - 1

### Seleccionamos aleatoriamente combinaciones de parámetros


In [79]:
best_params = None
best_return = -np.inf

In [80]:
for _ in range(5):  # 5 pruebas aleatorias
    sma_1 = np.random.choice(sma_1_range)
    sma_2 = np.random.choice(sma_2_range)
    
    total_return = backtest_strategy(sma_1, sma_2)
    print(f'Probando SMA_1: {sma_1}, SMA_2: {sma_2} - Rendimiento Total: {total_return:.2%}')
    
    if total_return > best_return:
        best_return = total_return
        best_params = {'SMA_1': sma_1, 'SMA_2': sma_2}

print(f'Los mejores parámetros son: {best_params} con un rendimiento total de {best_return:.2%}')

Probando SMA_1: 10, SMA_2: 250 - Rendimiento Total: 44.17%
Probando SMA_1: 90, SMA_2: 100 - Rendimiento Total: -1.35%
Probando SMA_1: 50, SMA_2: 150 - Rendimiento Total: 245.38%
Probando SMA_1: 70, SMA_2: 150 - Rendimiento Total: 208.27%
Probando SMA_1: 60, SMA_2: 100 - Rendimiento Total: 94.81%
Los mejores parámetros son: {'SMA_1': 50, 'SMA_2': 150} con un rendimiento total de 245.38%
